# 基于langchain创建自己专属的对话大模型

1. 领域精准问答
2. 数据更新频繁
3. 生成内容可解释可追溯
4. 数据隐私保护

通过这个例子，我们将基于`LangChain`, `OpenAI(LLM)`,  `vector DB`构建一个属于自己的LLM模型。

主要使用的技术————***Retrieval Augmented Generation (RAG)***

首先确保自己拥有一个 `OpenAI API key` (也并非必须)


### 准备环境

In [2]:
! pip install -qU \
    langchain==0.0.316 \
    openai==0.28.1  \
    tiktoken==0.5.1  \
    cohere \
    chromadb==0.4.15

ERROR: Could not install packages due to an OSError: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\blackink\\.conda\\envs\\langchain\\Lib\\site-packages\\langchain\\tools\\wikipedia\\__init__.py'
Consider using the `--user` option or check the permissions.



### 创建一个对话模型(no RAG)

In [8]:
from ragutils import ChatGpt
from langchain_openai import ChatOpenAI
import importlib

In [2]:
chatClient = ChatGpt()
chatClient.pushUserMessage('你好')
chatClient.pushSystemMessage('你好！有什么我可以帮助你的吗？')
chatClient.pushUserMessage('我的上一句话说了什么，以及你回答了什么？')


In [3]:
response = chatClient.getGptMsg()
print (response)

你说的是“你好”，而我回复你“你好！有什么我可以帮助你的吗？”如果你还有其他问题或者需要帮助，请告诉我！


OpenAI Python 的例子
```python
[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Knock knock."},
    {"role": "assistant", "content": "Who's there?"},
    {"role": "user", "content": "Orange."},
]
```
https://cookbook.openai.com/examples/how_to_format_inputs_to_chatgpt_models


但是langchain 需要使用以下的格式

In [9]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)


langchain_messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Knock knock."),
    AIMessage(content="Who's there?"),
    HumanMessage(content="Orange"),
]

print(langchain_messages)




[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Knock knock.', additional_kwargs={}, response_metadata={}), AIMessage(content="Who's there?", additional_kwargs={}, response_metadata={}), HumanMessage(content='Orange', additional_kwargs={}, response_metadata={})]


In [11]:
langchinChat = ChatOpenAI(
    openai_api_key="sk-OAbZ2yRJdYz9KOC207C60e1644274dCbB029B417E246E851",
    model='gpt-3.5-turbo',
    base_url="https://api3.apifans.com/v1"
)
res = langchinChat(langchain_messages).content
res

'Orange who?'

#### 处理LLM存在的缺陷
1. 容易出现幻觉
2. 信息滞后
3. 专业领域深度知识匮乏


In [12]:
messages = [
    SystemMessage(content="你是一个专业的知识助手。"),
    HumanMessage(content="你知道baichuan2模型吗？"),
]

In [6]:
res = langchinChat(messages)
print(res.content)


是的，我了解baichuan2模型。baichuan2是一种基于深度学习的图像分割模型，主要用于图像语义分割任务。它采用了编码-解码的架构，通过卷积神经网络提取图像特征，并将特征映射到与输入图像大小相同的分割结果。baichuan2模型在许多图像分割比赛和任务中取得了很好的性能，并被广泛应用于医学图像分割、自动驾驶、图像编辑等领域。


chatgpt AI无法满足我们在某些特定领域的专业需求，我们可以通过知识注入的方式，利用prompt来解决这种问题：

In [7]:

baichuan2_information = [
    "Baichuan 2是一个大规模多语言语言模型，它专注于训练在多种语言中表现优异的模型，包括不仅限于英文。这使得Baichuan 2在处理各种语言的任务时能够取得显著的性能提升。",
    "Baichuan 2是从头开始训练的，使用了包括了2.6万亿个标记的庞大训练数据集。相对于以往的模型，Baichuan 2提供了更丰富的数据资源，从而能够更好地支持多语言的开发和应用。",
    "Baichuan 2不仅在通用任务上表现出色，还在特定领域（如医学和法律）的任务中展现了卓越的性能。这为特定领域的应用提供了强有力的支持。"
]

source_knowledge = "\n".join(baichuan2_information)

In [8]:
print(source_knowledge)


Baichuan 2是一个大规模多语言语言模型，它专注于训练在多种语言中表现优异的模型，包括不仅限于英文。这使得Baichuan 2在处理各种语言的任务时能够取得显著的性能提升。
Baichuan 2是从头开始训练的，使用了包括了2.6万亿个标记的庞大训练数据集。相对于以往的模型，Baichuan 2提供了更丰富的数据资源，从而能够更好地支持多语言的开发和应用。
Baichuan 2不仅在通用任务上表现出色，还在特定领域（如医学和法律）的任务中展现了卓越的性能。这为特定领域的应用提供了强有力的支持。


In [ ]:
query = "你知道baichuan2模型吗？"

prompt_template = f"""基于以下内容回答问题：

内容:
{source_knowledge}

Query: {query}"""

In [ ]:
prompt = HumanMessage(
    content=prompt_template
)
messages.append(prompt)

res = chat(messages)

In [ ]:
print(res.content)


当我们注入一些专业的知识后，模型就能够很好的回答相关问题。
如果每一个问题都去用相关的外部知识进行增强拼接的话，那么回答的准确性就大大增加？？？？

### 创建一个RAG对话模型

#### 1. 加载数据 （以baichuan2论文为例）

   https://arxiv.org/pdf/2309.10305v2.pdf

In [9]:
! pip install pypdf

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 kB 2.2 MB/s eta 0:00:00a 0:00:01


In [11]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("Baichuan2.pdf")

pages = loader.load_and_split()

In [12]:
pages[0]

Document(page_content='Baichuan 2: Open Large-scale Language Models\nAiyuan Yang, Bin Xiao, Bingning Wang, Borong Zhang, Chao Yin, Chenxu Lv, Da Pan\nDian Wang, Dong Yan, Fan Yang, Fei Deng, Feng Wang, Feng Liu, Guangwei Ai\nGuosheng Dong, Haizhou Zhao, Hang Xu, Haoze Sun, Hongda Zhang, Hui Liu, Jiaming Ji\nJian Xie, Juntao Dai, Kun Fang, Lei Su, Liang Song, Lifeng Liu, Liyun Ru, Luyao Ma\nMang Wang, Mickel Liu, MingAn Lin, Nuolan Nie, Peidong Guo, Ruiyang Sun\nTao Zhang, Tianpeng Li, Tianyu Li, Wei Cheng, Weipeng Chen, Xiangrong Zeng\nXiaochuan Wang, Xiaoxi Chen, Xin Men, Xin Yu, Xuehai Pan, Yanjun Shen, Yiding Wang\nYiyu Li, Youxin Jiang, Yuchen Gao, Yupeng Zhang, Zenan Zhou, Zhiying Wu\nBaichuan Inc.\nAbstract\nLarge language models (LLMs) have\ndemonstrated remarkable performance on\na variety of natural language tasks based\non just a few examples of natural language\ninstructions, reducing the need for extensive\nfeature engineering. However, most powerful\nLLMs are closed-source

#### 2. 知识切片 将文档分割成均匀的块。每个块是一段原始文本

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50,
)

docs = text_splitter.split_documents(pages)

In [ ]:
len(docs)



#### 3. 利用embedding模型对每个文本片段进行向量化，并储存到向量数据库中

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma


embed_model = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents=docs, embedding=embed_model , collection_name="openai_embed")

#### 4. 通过向量相似度检索和问题最相关的K个文档。

In [ ]:
query = "How large is the baichuan2 vocabulary?"
result = vectorstore.similarity_search(query ,k = 2)

In [ ]:
result

#### 5. 原始`query`与检索得到的文本组合起来输入到语言模型，得到最终的回答

In [ ]:
def augment_prompt(query: str):
  # 获取top3的文本片段
  results = vectorstore.similarity_search(query, k=3)
  source_knowledge = "\n".join([x.page_content for x in results])
  # 构建prompt
  augmented_prompt = f"""Using the contexts below, answer the query.

  contexts:
  {source_knowledge}

  query: {query}"""
  return augmented_prompt

In [ ]:
print(augment_prompt(query))

In [ ]:
# 创建prompt
prompt = HumanMessage(
    content=augment_prompt(query)
)

messages.append(prompt)

res = chat(messages)

print(res.content)

### 没有OPENAI api key怎么办 创建一个非openai的对话模型  


1.   embedding模型  
2.   chat模型


In [ ]:
! pip install sentence-transformers

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

model_name = "sentence-transformers/sentence-t5-large"

In [ ]:
embedding = HuggingFaceEmbeddings(model_name=model_name)

In [ ]:
vectorstore_hf = Chroma.from_documents(documents=docs, embedding=embedding , collection_name="huggingface_embed")

In [ ]:
result = vectorstore_hf.similarity_search(query ,k = 2)


In [ ]:
print(result)


通过本地部署的模型进行交互